In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
os.chdir('./drive/MyDrive/RowingDATA/')

In [ ]:
os.listdir('./')

['Video1A.mp4',
 'Video1H.mp4',
 'Video2A.mp4',
 'Video2H.mp4',
 '.DS_Store',
 'rowing_csvs_out',
 'rowing_images_out',
 'Video1H-output.mp4',
 'Video1A-output.mp4',
 'rowing_dataset',
 'Video3A.mp4',
 'Video3F.mp4',
 'Video4A.mp4',
 'Video4F.mp4',
 'Video4A-output.mp4',
 'Video4F-output.mp4',
 'Video4F_values.csv',
 'Video3F_values.csv',
 'Video3F-output.mp4',
 'rowing_csvs_out_basic.csv',
 'Video3A-output.mp4',
 'Video3A_values.csv',
 'Video4A_values.gsheet',
 'Test3A.csv',
 'Test3F.csv',
 'Video4A_values.csv']

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

In [ ]:
def compute_similarity_score(weights, mse_normalized, pearson_corr, area_normalized):
    return (
        weights[0] * (1 - mse_normalized) +  # MSE contribution
        weights[1] * pearson_corr +          # Pearson correlation contribution
        weights[2] * (1 - area_normalized)   # Area Difference contribution
    )

In [ ]:
curve_files = ['Test3A.csv', 'Test3F.csv', 'Video4A_values.csv', 'Test3A.csv']  # Add all file paths
curves = [pd.read_csv(file).iloc[:, 1].values for file in curve_files]  # Extract y-values
x_points = np.linspace(0, len(curves[0]) - 1, len(curves[0]))  # Uniform x-axis

In [ ]:
# Define weights
weights = [0.2, 0.5, 0.3]  # Adjust as needed

# Compute pairwise similarity scores
n_curves = len(curves)
similarity_matrix = np.zeros((n_curves, n_curves))

for i in range(n_curves):
    for j in range(i + 1, n_curves):
        # Interpolate curves to the same x-points
        y1 = curves[i]
        y2 = np.interp(x_points, np.linspace(0, len(curves[j]) - 1, len(curves[j])), curves[j])

        # Compute metrics
        mse = np.mean((y1 - y2) ** 2)
        mse_max = max(y1.max() ** 2, y2.max() ** 2)
        mse_normalized = mse / mse_max

        pearson_corr, _ = pearsonr(y1, y2)

        area1 = np.trapz(y1, x_points)
        area2 = np.trapz(y2, x_points)
        area_difference = abs(area1 - area2)
        area_max = max(area1, area2)
        area_normalized = area_difference / area_max

        # Compute similarity score
        score = compute_similarity_score(weights, mse_normalized, pearson_corr, area_normalized)

        # Store in matrix (symmetric)
        similarity_matrix[i, j] = score
        similarity_matrix[j, i] = score

for i in range(n_curves):
    similarity_matrix[i, i] = 1  # Self-similarity is 1

In [ ]:
k = 2  # Number of lowest scores to consider
k_lowest_similarity_means = []
for i in range(n_curves):
    pairwise_scores = np.delete(similarity_matrix[i], i)  # Exclude diagonal
    k_lowest_scores = np.sort(pairwise_scores)[:k]  # Select k smallest scores
    k_lowest_mean = np.mean(k_lowest_scores)  # Compute mean of the k lowest scores
    k_lowest_similarity_means.append(k_lowest_mean)

least_synchronized_curve = np.argmin(k_lowest_similarity_means)

# Output results
print("Pairwise Similarity Matrix:")
print(similarity_matrix)
print(f"\nMean of {k} Lowest Similarity Scores (Lower is Worse):")
for i, score in enumerate(k_lowest_similarity_means):
    print(f"Curve {i + 1}: {score:.4f}")
print(f"\nCurve needing most adjustment: {least_synchronized_curve + 1}")

Pairwise Similarity Matrix:
[[1.         0.90398899 0.40409148 1.        ]
 [0.90398899 1.         0.31506177 0.90398899]
 [0.40409148 0.31506177 1.         0.40409148]
 [1.         0.90398899 0.40409148 1.        ]]

Mean of 2 Lowest Similarity Scores (Lower is Worse):
Curve 1: 0.6540
Curve 2: 0.6095
Curve 3: 0.3596
Curve 4: 0.6540

Curve needing most adjustment: 3
